In [5]:
# import time
# time.sleep(5000)

In [1]:
competition_name = "deep-learning-spring-2025-project-1"

### Import packages

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import optuna
import numpy as np
import pandas as pd
from datetime import datetime
import os

from data_loader import get_cifar10_dataloaders, get_test_dataloader
from trainer import train_model
from model import ResNet18

/scratch/gk2657/DLSP25-Project1/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configure the device

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [4]:
# See the total number of trainable parameters
def num_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [5]:
model = ResNet18()
num_params(model)

4903242

In [6]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    num_epochs = trial.suggest_int("num_epochs", 10, 50)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    momentum = trial.suggest_uniform("momentum", 0.5, 0.9) if optimizer_name == "SGD" else None

    # Suggest data transformations
    transform = transforms.Compose([
        # add random crop and padding
        transforms.RandomHorizontalFlip(trial.suggest_float("h_flip", 0.0, 1.0)),
        transforms.RandomRotation(trial.suggest_int("rotation", 0, 30)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalize with mean and std of CIFAR-10
    ])
    
    train_loader, valid_loader = get_cifar10_dataloaders(
        transform,
        subset_percent=1.0, 
        valid_size=0.1,
        batch_size=batch_size,
        num_workers=4
    )

    # Define model
    model = ResNet18()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model_name = "resnet"

    # Print the current hyperparameters, transformations, and model name
    print("-" * 50)
    print(f"Trial {trial.number}:")
    print(f"Model: {model_name}")
    print(f"Trainable Parameters: {num_params}")
    print(f"Epochs: {num_epochs}")
    print(f"Batch Size: {batch_size}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Optimizer: {optimizer_name}")
    if optimizer_name == "SGD":
        print(f"Momentum: {momentum}")
    print(f"Transformations: {transform}")
    print("- " * 25)

    # Define optimizer
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss()

    # Training
    best_val_accuracy = train_model(
        trial, model, train_loader, criterion, optimizer, 
        valid_loader=valid_loader, num_epochs=num_epochs, device=device)
    
    # Checkpoint the model with the best validation accuracy
    model_filename = f"model_trial_{trial.number}_val_acc_{best_val_accuracy:.4f}.pth"
    study_name = trial.study.study_name
    checkpoint_dir = f"checkpoints_{study_name}"
    model_path = os.path.join(checkpoint_dir, model_filename)
    
    # Create a directory for checkpoints if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Save the model state_dict
    torch.save(model.state_dict(), model_path)
    print(f"Model checkpoint saved to {model_path}")

    return best_val_accuracy

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
study_name = f"study_{timestamp}"

study = optuna.create_study(direction="maximize", study_name=study_name)
study.optimize(objective, n_trials=20)

print("Best trial:", study.best_trial.number)
print("Best hyperparameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

[I 2025-03-09 12:42:46,559] A new study created in memory with name: study_2025-03-09_12-42-46


--------------------------------------------------
Trial 0:
Model: resnet
Trainable Parameters: <function num_params at 0x14f0a0a4b940>
Epochs: 46
Batch Size: 256
Learning Rate: 0.005965107711939086
Optimizer: RMSprop
Transformations: Compose(
    RandomHorizontalFlip(p=0.06640679932192928)
    RandomRotation(degrees=[-29.0, 29.0], interpolation=nearest, expand=False, fill=0)
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
)
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/46], Batch [100/176], Loss: 1.7964
  Validation Accuracy after Epoch 1: 0.3694
  Epoch [2/46], Batch [100/176], Loss: 1.5153
  Validation Accuracy after Epoch 2: 0.4332
  Epoch [3/46], Batch [100/176], Loss: 1.2579
  Validation Accuracy after Epoch 3: 0.5672
  Epoch [4/46], Batch [100/176], Loss: 1.0908
  Validation Accuracy after Epoch 4: 0.5522
  Epoch [5/46], Batch [100/176], Loss: 1.0960
  Validation Accuracy after Epoch 5: 0.5778
  Epoch [6/46], Batch [100/17

### Load Checkpoint

In [27]:
model = ResNet18()
model.to(device)

# Load the latest checkpoint
checkpoint_dir = f"checkpoints_{study_name}"
checkpoint = torch.load(f"{checkpoint_dir}/model_trial_0_val_acc_0.0000.pth")
model.load_state_dict(checkpoint)

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints_study_2025-03-09_12-31-02/model_trial_0_val_acc_0.0000.pth'

In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalize with mean and std of CIFAR-10
])

from trainer import evaluate_model
_, valid_loader = get_cifar10_dataloaders(
    transform,
    subset_percent=0.0002, 
    valid_size=0.1,
    batch_size=128,
    num_workers=4
)

acc = evaluate_model(model, valid_loader, device)
print("Acc:", acc)

Image shape: torch.Size([3, 32, 32])
Label: 8
Number of training data: 9
Number of validation data: 1
Acc: 0.0


### Run model on test data

In [ ]:
# Generate submission file with test data
test_loader = get_test_dataloader()

model.eval()
predictions = []

with torch.no_grad():
    for images, in test_loader:
        outputs = model(images) 
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

submission = pd.DataFrame({'ID': np.arange(len(predictions)), 'Labels': predictions})
submission.to_csv('submission.csv', index=False)
print("submission file saved.")

In [23]:
# import kaggle
# kaggle.api.competition_submit(
#     file_name="submission.csv",
#     message="test",
#     competition=competition_name
# )